In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Bidirectional
from tensorflow.keras.models import Model
import re
import unicodedata

In [2]:
# Get the current working directory
cwd = os.getcwd()

# Join the paths
tokenizer_path = os.path.join(cwd, 'data_dd', 'tokenizer_dd.pickle')
model_path = os.path.join(cwd, 'data_dd', 's2s_model_dd.h5')

# Load tokenizer from pickle
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
# Define model parameters
max_length = 15 # Max words in string or sequence
latent_dim = 200
num_encoder_tokens = len(tokenizer.word_index) + 1
num_decoder_tokens = len(tokenizer.word_index) + 1
learning_rate = 0.001

# Define encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
# Make the LSTM layer bidirectional
encoder_lstm = Bidirectional(LSTM(latent_dim, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Update latent_dim to match the concatenated states
latent_dim *= 2

# Define decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
decoder_embedded = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Load the model
model = load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 200)    3077000     ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 400),        641600      ['embedding[0][0]']              
                                 (None, 200),                                                 

In [4]:
# Define encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Define decoder model for inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inference_inputs = Input(shape=(None,))
decoder_embedding_inference = decoder_embedding(decoder_inference_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_inference, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inference_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

# Save token index mappings
target_token_index = tokenizer.word_index
reverse_target_token_index = {v: k for k, v in target_token_index.items()}

In [5]:
# Contraction mapping
contractions = {
    "’": "'",
    "‘": "'",
    "“": '"',
    "”": '"',
    "can't": "cannot",
    "won't": "will not",
    "n't": " not",
    "i'm": "i am",
    "i'd": "i would",
    "thats's": "that is",
    "it's": "it is",
    "he's": "he is",
    "she's": "she is",
    "you're": "you are",
    "they're": "they are",
    "we're": "we are",
    "i've": "i have",
    "you've": "you have",
    "they've": "they have",
    "we've": "we have",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "doesn't": "does not",
    "don't": "do not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
    "wouldn't": "would not",
    "shouldn't": "should not",
    "couldn't": "could not",
    "mightn't": "might not",
    "mustn't": "must not",
    "she'd": "she would",
    "he'd": "he would",
    "they'd": "they would",
    "we'd": "we would",
    "that'll": "that will",
    "there'll": "there will",
    "who'll": "who will",
    "it'll": "it will",
    "that'd": "that would",
    "there'd": "there would",
    "who'd": "who would",
    "when's": "when is",
    "where's": "where is",
    "why's": "why is",
    "how's": "how is",
    "y'all": "you all",
    "let's": "let us",
    "ma'am": "madam",
    "o'clock": "of the clock",
    "ain't": "is not",
    "could've": "could have",
    "should've": "should have",
    "would've": "would have",
    "might've": "might have",
    "must've": "must have",
    "who've": "who have",
    "oughtn't": "ought not",
    "daren't": "dare not",
    "needn't": "need not",
    "what's": "what is",
    "usedn't": "used not"
}

def normalize_text(text: str) -> str:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = text.lower()
    text = re.sub(r"\s*'\s*", "'", text)
    text = re.sub(r"\s*([.!?])\s*", r" \1 ", text)
    for contraction, replacement in contractions.items():
        text = re.sub(re.escape(contraction), replacement, text)
    text = re.sub(r"[^a-z' ]", ' ', text)
    text = re.sub(r"\s+", ' ', text).strip()
    return text

def preprocess_text(text: str) -> str:
    text = normalize_text(text)
    words = text.split()[:max_length]
    trimmed_text = ' '.join(words)
    return trimmed_text

def generate_response(input_seq: np.ndarray, max_decoder_seq_length: int) -> str:
    states_value = encoder_model.predict(input_seq, verbose=0)
    target_seq = np.ones((1, 1)) # Was np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['<START>']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        if (sampled_char == '<END>' or len(decoded_sentence.split()) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.ones((1, 1)) # Was np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence.strip().replace('<START>', '').replace('<END>', '').strip()

In [ ]:
def chat():
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        input_text = preprocess_text(user_input)
        input_sequence = [tokenizer.texts_to_sequences([input_text])[0]]
        padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length, padding='pre', truncating='post')
        response = generate_response(np.array(padded_input_sequence), max_length)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    chat()

Chatbot is ready! Type 'exit' to end the conversation.


You:  Hello


Chatbot: gold dresses unchanged recommend recommend protons protons prepared acrobatics acrobatics spraining saw spraining finish cigar advised


You:  what is your name?


Chatbot: rigorous mishap smoker tumion sizeable reader mag drizzle gao collide existing ahhhhhhhh traveled buff touched instrument
